<a href="https://colab.research.google.com/github/rodrigoms95/herramientas-climatico-22-1/blob/main/code/Proyecto/Jupyter/sensibilidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Activar Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import os

import pandas as pd
import numpy as np

import xgboost as xgb

from matplotlib import pyplot as plt

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
path_drive = "/content/drive/MyDrive/Colab/Subsidio_electricidad/" 
path_data = path_drive + "data/Actual/data"
path_r = path_drive + "Graficas/"
csv = ".csv"

# Escenarios
RCP = ["RCP4p5", "RCP8p5"]
fut = [2030, 2050, 2070]
path_data = path_drive + "data/" + RCP[1] + "/data."
path_d = path_drive + "data/Actual/data"
path_cfe = path_drive + "CFE/"

# Cantidad de corridas.
s = 100

cols = ["trains_X", "trains_Y", "tests_X", "tests_Y", "U_preds", "C_preds"]

# Crear carpetas.
path_m = path_drive + "Modelos/"
if not os.path.exists(path_m):
    os.mkdir(path_m)
for c in ( cols + ["U_model", "C_model", str(fut[0]), "Pago"] ):
    if not os.path.exists(path_m + c + "/"):
        os.mkdir(path_m + c + "/")

In [ ]:
# Entrena un modelo de Gradient Boosted Trees
# para predecir el consumo eléctrico
# y la cantidad de usuarios.

# Entrena el modelo final utilizado para el paper.
data_0 = pd.read_csv(path_d + csv)
# Unimos las tarifas 1 y DAC.
data_0["Consumo_1*"] += data_0["Consumo_DAC"]
data_0["Usuarios_1*"] += data_0["Usuarios_DAC"]
data_0.drop( ["Consumo_DAC", "Usuarios_DAC"], axis = 1, inplace = True )
# Escogemos las variables relevantes.
data_0.drop( ["Area", "CVE_MUN", "Tmax", "Tmin", "HDD_mean", "HDD_p10",
    "CDD_p90", "$luz", "Tmean_max_2", "M_verano", "Verano"],
    axis = 1, inplace = True)
# Creamos columnas de apoyo.
data_0["Consumo_Usuario"] = data_0["Consumo_1*"] / data_0["Usuarios_1*"]
data_0["Usuario_poblacion"] = data_0["Usuarios_1*"] / data_0["Poblacion"]
# Límites para corrección.
lim = [0.1, 0.6, 30] 
# Estadísticas de corrección.
# Porcentaje de municipios con corrección.
a = data_0.loc[ data_0["Usuario_poblacion"] < lim[0] ].shape[0]
b = data_0.loc[ data_0["Usuario_poblacion"] > lim[1] ].shape[0]
c = data_0.loc[ data_0["Consumo_Usuario"]   < lim[2] ].shape[0]
d = ( a + b + c ) / 12 / 7 / 2469 * 100
# Porcentaje de población con corrección.
a = data_0.loc[ data_0["Usuario_poblacion"] < lim[0], "Poblacion" ].sum()
b = data_0.loc[ data_0["Usuario_poblacion"] > lim[1], "Poblacion" ].sum()
c = data_0.loc[ data_0["Consumo_Usuario"]   < lim[2], "Poblacion" ].sum()
d = ( a + b + c ) / 12 / 7 / data_0["Poblacion"].sum() * 100
# Porcentaje de consumo con corrección.
a = data_0.loc[ data_0["Usuario_poblacion"] < lim[0], "Consumo_1*" ].sum()
b = data_0.loc[ data_0["Usuario_poblacion"] > lim[1], "Consumo_1*" ].sum()
c = data_0.loc[ data_0["Consumo_Usuario"]   < lim[2], "Consumo_1*" ].sum()
d = ( a + b + c ) / 12 / 7 / data_0["Poblacion"].sum() * 100
# Correcciones de municipios que están abajo o arriba de los límites.
data_0.loc[data_0["Usuario_poblacion"] < lim[0], "Usuarios_1*"] = (
    data_0.loc[data_0["Usuario_poblacion"] < lim[0], "Poblacion"] * lim[0] )
data_0.loc[data_0["Usuario_poblacion"] > lim[1], "Usuarios_1*"] = (
    data_0.loc[data_0["Usuario_poblacion"] > lim[1], "Poblacion"] * lim[1] )
data_0.loc[data_0["Consumo_Usuario"] < lim[2], "Consumo_1*"] = (
data_0.loc[data_0["Consumo_Usuario"] < lim[2], "Usuarios_1*"] * 10 )
data_0["Consumo_Usuario"] = data_0["Consumo_1*"] / data_0["Usuarios_1*"]
data_0["Usuario_poblacion"] = data_0["Usuarios_1*"] / data_0["Poblacion"]
# Consumo y usuarios totales en 2010.
cons = data_0[data_0["Año"] == 2010]['Consumo_1*'].sum()
us = data_0[data_0["Año"] == 2010].groupby('CVE_INEGI'
    ).mean()['Usuarios_1*'].sum()

In [ ]:
# Entrenamiento de modelos.

# Hiperparámetros.
params = {
    "objective": "reg:squarederror",
    "colsample_bytree": 0.3,
    "learning_rate": 0.1,
    "max_depth": 50,
    "min_child_weight" : 25,
    "min_split_loss": 0.2,
    "n_estimators": 100
    }

# Datos de usuarios.
# Escogemos el conjunto de features y de variables a predecir.
U_Y = data_0.iloc[:, 5]
U_X = data_0[ ["Poblacion", "PIB", "PCI", "Año"] ].copy()
U_X = pd.concat( (U_X, pd.get_dummies(data_0["CVE_ENT"]) ), axis = 1)

# Datos de consumo por usuario.
# Escogemos el conjunto de features y de variables a predecir.
C_X, C_Y = data_0.iloc[:, 7:-2], data_0.iloc[:, -2]
# Quitamos algunas features.
C_X.drop( ["Pre", "Año", "Poblacion", "PIB", "lat", "lon"],
    axis = 1, inplace = True )
C_X = pd.concat( (C_X, pd.get_dummies(data_0["CVE_ENT"]) ), axis = 1)

# Calculamos el RMSE para todas las corridas.
# Separamos en conjuntos de entrenamiento y de prueba.
for i in range( 1, s + 1 ):
    if i % 10 == 0: print(f"Entrenando modelo {i}...")

    # Inicializamos valores.
    trains_X = None
    trains_Y = None
    tests_X  = None
    tests_Y  = None
    U_preds  = None
    U_RMSE   = None
    U_R2     = None
    C_preds  = None
    C_RMSE   = None
    C_R2     = None
    U_reg    = None
    C_reg    = None

    # Predicción de usuarios.
    # Separamos en conjuntos de entrenamiento y de prueba.
    trains_X, tests_X, trains_Y, tests_Y = train_test_split(
        U_X, U_Y, test_size = 0.3)
    # Creamos el regresor con los hiperparámetros.
    U_reg = xgb.XGBRegressor( **params )
    # Entrenamos el modelo.
    U_reg.fit(trains_X, trains_Y)
    # Probamos la regresión en el set de entrenamiento.
    U_preds = pd.DataFrame( U_reg.predict( tests_X ), columns = ["Preds"] )
    # Calculamos el error de entrenamiento.
    U_RMSE = np.sqrt( mean_squared_error( tests_Y, U_preds ) )
    U_R2   = ( r2_score( tests_Y, U_preds) - 0.999 ) * 10000

    # Predicción de consumo por usuario.
    # Separamos en conjuntos de entrenamiento y de prueba.
    trains_X, tests_X, trains_Y, tests_Y = train_test_split(
        C_X, C_Y, test_size = 0.3)
    # Creamos el regresor con los hiperparámetros.
    C_reg = xgb.XGBRegressor( **params )
    # Entrenamos el modelo.
    C_reg.fit(trains_X, trains_Y)
    # Probamos la regresión en el set de entrenamiento.
    C_preds = pd.DataFrame( C_reg.predict( tests_X ), columns = ["Preds"] )
    # Calculamos el error de entrenamiento.
    C_RMSE = np.sqrt( mean_squared_error( tests_Y, C_preds ) )
    C_R2   = ( r2_score( tests_Y, C_preds ) - 0.95 ) * 1000

    resultados = [ trains_X, trains_Y, tests_X, tests_Y, U_preds, C_preds ]

    # Guardamos la información generada por cada modelo.
    for j in range( len(resultados) ):
        resultados[j].to_csv( path_m + cols[j] + "/"
            + cols[j]  + "_" + str(i) + ".csv" )

    # Agregamos los estadísticos del modelo a un dataframe.
    stats_c = ["U_RMSE", "U_R2", "C_RMSE", "C_R2"]
    if i == 1: df = pd.DataFrame( columns = stats_c )
    else     :  df = pd.read_csv(path_m + "stats.csv", index_col = 0)
    df_1 = pd.DataFrame( [[U_RMSE, C_R2, C_RMSE, C_R2]], index = [i],
        columns = stats_c )
    pd.concat( [df, df_1] ).to_csv( path_m + "stats.csv" )

    # Guardamos los modelos
    U_reg.save_model(path_m + "U_model/U_model_" + str(i))
    C_reg.save_model(path_m + "C_model/C_model_" + str(i))

Entrenando modelo 90...
Entrenando modelo 100...


In [ ]:
# Predicciones futuras.

# Datos.
fname = "data.csv"
# Directorios.
f_path = path_data + str(fut[0]) + csv

# Agregamos las columnas a evaluar.
df = pd.read_csv(path_m + "stats.csv", index_col = 0)
cols = [ "m_u", "p_u", "c_u", "c_a", "m_b", "perc_c", "perc_u" ]
df[ cols ] = None
df.to_csv( path_m + "stats.csv" )

# Iteramos para todos los modelos.
for i in range( 1, s + 1 ):
    if i % 10 == 0: print(f"Procesando modelo {i}...")

    # Cargamos los modelos.
    U_reg = xgb.XGBRegressor()
    C_reg = xgb.XGBRegressor()
    U_reg.load_model(path_m + "U_model/U_model_" + str(i))
    C_reg.load_model(path_m + "C_model/C_model_" + str(i))

    # Cargamos la información futura.
    data = pd.read_csv(f_path).copy()
    data["Año"] = 2030

    # Escogemos el conjunto de features para
    # la predicción de consumo por usuario.
    C_X = data[ ["Tmean", "CDD_mean", "Pre_Tmean",
        "Densidad_poblacion", "PCI", "$GLP", "Mes"] ].copy()
    C_X = pd.concat( (C_X, pd.get_dummies(data["CVE_ENT"]) ), axis = 1)

    # Escogemos el conjunto de features para la predicción de usuarios.
    U_X = data[ ["Poblacion", "PIB", "PCI", "Año"] ].copy()
    U_X = pd.concat( (U_X, pd.get_dummies(data["CVE_ENT"]) ), axis = 1)
    
    # Hacemos la predicción y calculamos el consumo total.
    data["Consumo_Usuario"] = C_reg.predict(C_X)
    data["Usuarios_1*"] = U_reg.predict(U_X)
    data["Consumo_1*"] = data["Usuarios_1*"] * data["Consumo_Usuario"]
    data["Usuario_poblacion"] = data["Usuarios_1*"] / data["Poblacion"]
    
    # Nombre de la corrida.
    # Límites para corrección.
    lim = [0.11, 1.6, 30] 

    # Estadísticas de corrección.
    # Porcentaje de municipios abajo o arriba del umbral.
    m_u = ( ( data[ data["Usuario_poblacion"] < lim[0] ].shape[0]
        +     data[ data["Usuario_poblacion"] > lim[1] ]["Poblacion"
        ].shape[0] ) / 12 / 2649 * 100 )
    # Porcentaje de la población abajo o arriba del umbral.
    p_u = ( ( data.loc[ data["Usuario_poblacion"] < lim[0], "Poblacion" ].sum()
        + data.loc[ data["Usuario_poblacion"] > lim[1],
        "Poblacion"].sum() ) / data["Poblacion"].sum() * 100 )
    # Porcentaje del consumo abajo o arriba del umbral.
    c_u = ( ( data_0.where( data["Usuario_poblacion"] > lim[1]
        ).dropna().groupby('CVE_INEGI').mean()["Consumo_1*"].sum() + 
        ( data_0.where( data["Usuario_poblacion"] < lim[0] ).dropna()
        .groupby('CVE_INEGI').mean()["Consumo_1*"].sum() ) ) * 100 / cons )
    # Porcentaje del consumo arriba del umbral.
    c_a = data[ data["Usuario_poblacion"] > lim[1] ]
    c_a = ( ( ( c_a["Usuario_poblacion"] - lim[1] ) * c_a["Consumo_Usuario"]
        * c_a["Poblacion"] ).sum() / data["Consumo_1*"].sum() * 100 )
    # Porcentaje de municipios abajo del umbral de Consumo_Usuario.
    m_b = data[ data["Consumo_Usuario"] < lim[2] ].shape[0] / 12 / 2469 * 100

    # Correcciones de municipios que están abajo o arriba de los límites.
    data.loc[ data["Usuario_poblacion"] < lim[0], "Usuarios_1*" ] = (
        data.loc[ data["Usuario_poblacion"] < lim[0], "Poblacion" ] * lim[0] )
    data.loc[ data["Usuario_poblacion"] > lim[1], "Usuarios_1*" ] = (
        data.loc[ data["Usuario_poblacion"] > lim[1], "Poblacion" ] * lim[1] )
    data["Consumo_1*"] = ( data["Usuarios_1*"] * data["Consumo_Usuario"] )
    data.loc[ data["Consumo_Usuario"] < lim[2], "Consumo_1*" ] = (
        data.loc[ data["Consumo_Usuario"] < lim[2], "Usuarios_1*" ] * 10 )

    # Retiramos las columnas de apoyo.
    data.drop( columns = ["Consumo_Usuario", "Usuario_poblacion"],
        inplace = True )

    # Incremento en consumo con respecto a 2010.
    perc_c = ( data["Consumo_1*"].sum() / cons - 1 ) * 100
    # Incremento en usuarios con respecto a 2010.
    perc_u = data.groupby("CVE_INEGI").mean()["Usuarios_1*"].sum()
    perc_u = ( perc_u / us - 1 ) * 100

    # Agregamos la información calculada al dataframe.
    row = [ m_u, p_u, c_u, c_a, m_b, perc_c, perc_u ]
    df = pd.read_csv(path_m + "stats.csv", index_col = 0)
    df.loc[ i, cols ] = row
    df.to_csv( path_m + "stats.csv" )

    data.to_csv( path_m + str(fut[0]) + "/data."
        + str(fut[0]) + "_" + str(i) + ".csv" )

Procesando modelo 10...
Procesando modelo 20...
Procesando modelo 30...
Procesando modelo 40...
Procesando modelo 50...
Procesando modelo 60...
Procesando modelo 70...
Procesando modelo 80...
Procesando modelo 90...
Procesando modelo 100...


In [ ]:
# Calcula el pago de luz para tarifas residenciales.

# Directorios.
f_tarifa = "Tarifa_1.csv"
f_dac = "DAC.csv"
f_regiones = "Regiones.csv"

tarifa = pd.read_csv(path_cfe + f_tarifa)
dac = pd.read_csv(path_cfe + f_dac)
regiones = pd.read_csv(path_cfe + f_regiones)

# Tipos de escalones para las tarifas.
escalon_0 = tarifa["Escalon"].unique()

# Se establece la tarifa DAC para cada estado.
regiones[ ["DAC 04-2013", "Equilibrio"] ] = np.nan
for i in regiones.itertuples():
    regiones.loc[ i[0], ["DAC 04-2013", "Equilibrio"] ] = ( 
        dac.drop(8).loc[ dac["Region"] == i.Region,
        ["DAC 04-2013", "Equilibrio"] ].values[0] )

# Impuestos y cargos fijos.    
IVA = 1.16
FIJO_DAC = 78.85
MULT = 0.86

In [ ]:
# Pago futuro

names  = ["", "_tarifa_teorica", "pob_pib"]
precio = ["Precio 04-2013", "Precio Propuesto", "Precio Propuesto_2"]
limite = [        "Límite",         "Límite_p",         "Límite_p_2"]
p_real = ["Pago_real", "Pago_real_p", "Pago_real_p_2"]
p_sub  = ["Pago_sin_subsidio", "Pago_sin_subsidio_p", "Pago_sin_subsidio_p_2"]
sub_us = ["Subsidio_Usuario", "Subsidio_Usuario_p", "Subsidio_Usuario_p_2"]
sub    = ["Subsidio", "Subsidio_p", "Subsidio_p_2"]

# Agregamos las columnas a evaluar.
data = pd.read_csv(path_m + "stats.csv", index_col = 0)
n_cols = [ "Pago", "Subsidio" ]
cols = ( ["Usuarios", "Consumo"] + n_cols
    + [x + "_p" for x in n_cols ] + [x + "_p_2" for x in n_cols ] )
data[ cols ] = None
data.to_csv( path_m + "stats.csv" )
# Iteramos para todos los modelos.
for k in range( 1, s + 1 ):
    if k % 5 == 0: print(f"Procesando modelo {k}...")

    data = pd.read_csv(path_m + "stats.csv", index_col = 0)

    # Iteramos para todas las propuestas.
    for m in range( len(precio) ):

        f_path = [ path_m + str(fut[0]) + "/data." + str(fut[0]), 
            path_m + "Pago" + "/data." + str(fut[0]) + "_Pago" ]

        cons = ["Verano_Consumo_1*", "Verano_Consumo_DAC",
            "Invierno_Consumo_1*", "Invierno_Consumo_DAC"]

        # Información mensual.
        df_0 = pd.read_csv(f_path[0] + "_" + str(k) + csv)
        # Información anual.
        df = pd.read_csv(f_path[0] + "_yearly" + csv)
        df.set_index("CVE_INEGI", inplace = True)
        df["Usuarios_1*"] = df_0.groupby( "CVE_INEGI").mean()["Usuarios_1*"]

        # Redondeamos la cantidad de usuarios.
        df["Usuarios_1*"] = df["Usuarios_1*"].round()

        # Retiramos la columna de año, que ya no tiene sentido.
        df.drop(columns = "Año", inplace = True)

        # Sumamos el consumo correspondiente al verano.
        df_2 = df_0.where(df_0["Verano"] == True)
        df_2 = df_2.groupby(["CVE_INEGI"]).sum()
        df[cons[0]] = df_2["Consumo_1*"]

        # Sumamos el consumo correspondiente al invierno.
        df_2 = df_0.where(df_0["Verano"] == False)
        df_2 = df_2.groupby(["CVE_INEGI"]).sum()
        df[cons[2]] = df_2["Consumo_1*"]

        # Retiramos las columnas de consumo total anual.
        df.drop(columns = [ "Consumo_DAC" ], inplace = True)

        # Unimos las tarifas 1 y DAC.
        df.drop("Usuarios_DAC", axis = 1, inplace = True)

        df.reset_index(inplace = True)
        
        # Se crean columnas para los
        # escalones de verano e invierno.
        verano_escalon = []
        invierno_escalon = []
        for e in escalon_0:
            verano_escalon.append("Verano_" + e)
            invierno_escalon.append("Invierno_" + e)
        escalon = [verano_escalon, invierno_escalon]
        df[verano_escalon] = np.nan
        df[invierno_escalon] = np.nan
        
        # Se establece el valor de consumo límite para cada escalon.
        for i in df.itertuples():
            df.loc[i[0], invierno_escalon + verano_escalon] = tarifa.loc[
                tarifa["Tarifa"] == i.Tarifa, limite[m]].array * 6

        # Columnas para consumo en verano e invierno.
        estacion = ["Verano_", "Invierno_"]
        verano_escalon_cons = [ "Cons_" + x for x in verano_escalon ]
        invierno_escalon_cons = [ "Cons_" + x for x in invierno_escalon ]
        escalon_cons = [ verano_escalon_cons, invierno_escalon_cons ]

        # Columnas para verano.
        df["Verano_Consumo_Usuario"] = df["Verano_Consumo_1*"] / df["Usuarios_1*"]
        df[escalon_cons[0]] = df[escalon[0]].copy()
        df[estacion[0] + "Pago"] = 0

        # Columnas para invierno.
        df["Invierno_Consumo_Usuario"] = ( df["Invierno_Consumo_1*"]
            / df["Usuarios_1*"] )
        df[escalon_cons[1]] = df[escalon[1]].copy()
        df[estacion[1] + "Pago"] = 0

        df["DAC_Pago"] = 0

        # Iteramos para verano e invierno.
        for e in [0, 1]:

            # Offset de columnas de verano e invierno.
            if e == 0: t = 0
            else: t = 6

            col = np.concatenate( [ [estacion[e] + "Consumo_Usuario"],
                escalon_cons[e] ] )

            # Calculamos los escalones de consumo
            # de acuerdo con cada límite.
            for i in range(len(col[:-1])):
                df[col[i + 1]] = df[col[i]] - df[col[i + 1]]

            # Eliminamos valores negativos.
            df[col[1:]] = df[col[1:]].clip( lower = 0 )

            col = np.concatenate( [ escalon_cons[e] ] )
            col = np.flip(col)
    
            # Recorremos los escalones a su posición adecuada.
            for i in range(len(col[:-1])):
                df[col[i]] = df[col[i + 1]]

            for i in df.itertuples():
        
                # No se está en tarifa DAC.
                if ( ( i.Verano_Consumo_Usuario +
                    i.Invierno_Consumo_Usuario )
                    <= ( i.Verano_Excedente * 2 ) ):
                    # Si el consumo es menor al límite básico, el
                    # escalon básico es igual al consumo total.
                    if i[40 + t] == 0:
                        df.loc[i[0], escalon_cons[e][0]] = i[39 + t]
                    # Si el consumo excede un escalón, 
                    # el consumo de un escalón es su límite.
                    for j in range(3):
                        if i[40 + t + j + 1] > 0:
                            df.loc[i[0], escalon_cons[e][j]] = i[
                                31 + t * 4 // 6 + j]
                    
                    df.loc[i[0], escalon_cons[e]] *= tarifa.query(
                        "Tarifa=='" + i.Tarifa + "' & Temporada=='"
                        + estacion[e][:-1] + "'")[precio[m]].array
    
                # Sí está en tarifa DAC.
                else:
                    # El resto de pagos es igual a cero.
                    df.loc[i[0], col] = 0

                    # Se calcula el pago con todo el consumo general.
                    df.loc[i[0], "DAC_Pago"] = ( ( i.Verano_Consumo_Usuario +
                        i.Invierno_Consumo_Usuario ) * regiones.loc[
                        regiones["CVE_ENT"] == i.CVE_ENT,
                        "DAC 04-2013"].values[0] + FIJO_DAC )

            # Se calcula el pago por temporada.
            df[estacion[e] + "Pago"] = ( df[ np.concatenate( [ escalon_cons[e]
                ] ) ].sum(axis = 1) * IVA )

        # Se eliminan las columnas de apoyo.
        df.drop( columns = np.concatenate( [
            escalon_cons[0], escalon[0], escalon_cons[1], escalon[1],
            ["Verano_Consumo_Usuario", "Invierno_Consumo_Usuario"]
            ] ), inplace = True )

        # Se calcula el pago total.
        df[ p_real[m] ] = ( df[[estacion[0] + "Pago",
            estacion[1] + "Pago", "DAC_Pago"]].sum(axis = 1) )

        # Se eliminan los pagos parciales.
        df.drop( columns = [ estacion[0] + "Pago",
            estacion[1] + "Pago", "DAC_Pago" ], inplace = True )

        # Unimos el consumo de verano e invierno.
        df["Consumo_1*"] = ( df["Verano_Consumo_1*"]
            + df["Invierno_Consumo_1*"] )
        df["Consumo_Usuario"] = df["Consumo_1*"] / df["Usuarios_1*"]
        df[ p_sub[m] ] = np.nan

        # Calculamos el precio sin subsidio, es decir,
        # con un precio plano igual al precio marginal.
        for i in df.itertuples():
            df.loc[i[0],  p_sub[m] ] = ( i.Consumo_Usuario * regiones.loc[
                regiones["CVE_ENT"] == i.CVE_ENT,
                "Equilibrio" ].values[0] * IVA * MULT )

        # Se calcula el subsidio.
        df[ sub_us[m] ] = ( df[ p_sub[m] ] - df[ p_real[m] ] )
        df[ sub[m] ] = df[ sub_us[m] ] * df["Usuarios_1*"]

        df.to_csv( f_path[1] + "_" + str(k) + csv, index = False )

        # Agregamos la información calculada al dataframe.
        row = [ df[ p_real[m] ].sum(), df[ sub[m] ].sum() ]
        data.loc[ k, cols[ 3 + m * 2 : 5 + m * 2 ] ] = row
    
    row = [ df["Usuarios_1*"].sum(), df["Consumo_1*"].sum() ]
    data.loc[ k, cols[ 0:3 ] ] = row
    data.to_csv( path_m + "stats.csv" )

In [ ]:
# Estadísticas de modelos.
df = pd.read_csv(path_m + "stats.csv", index_col = 0)
df.describe().iloc[:, 11:]

,Subsidio,Usuarios,Consumo,Subsidio_p,Subsidio_p_2,Pago,Pago_p,Pago_p_2
count,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02,1.000000e+02
mean,1.311001e+11,4.637879e+07,7.676045e+10,6.833020e+10,3.848252e+10,2.968631e+06,4.986145e+06,6.288939e+06
std,6.282973e+08,1.798506e+05,4.145102e+08,3.161533e+08,2.510534e+08,1.032851e+04,2.520674e+04,2.699149e+04
min,1.295240e+11,4.584035e+07,7.570018e+10,6.754433e+10,3.788498e+10,2.946340e+06,4.923186e+06,6.226401e+06
25%,1.307872e+11,4.627560e+07,7.653209e+10,6.815671e+10,3.836305e+10,2.960338e+06,4.968946e+06,6.270412e+06
50%,1.311670e+11,4.637470e+07,7.679693e+10,6.830122e+10,3.846304e+10,2.969257e+06,4.985839e+06,6.287918e+06
75%,1.315695e+11,4.651008e+07,7.706490e+10,6.850068e+10,3.861995e+10,2.977250e+06,5.003020e+06,6.308783e+06
max,1.325090e+11,4.692900e+07,7.754875e+10,6.930374e+10,3.923504e+10,2.986929e+06,5.044515e+06,6.348193e+06


In [96]:
# Estadísticas de usuarios y población.
actual = pd.read_csv(path_d + csv)
actual["Cons_us"] = actual["Consumo_1*"] / actual["Usuarios_1*"]
stats_2 = actual.describe()

stats_2[["Usuarios_1*", "Poblacion", "PCI"]]

,Usuarios_1*,Poblacion,PCI
count,207396.000000,2.073960e+05,2.073960e+05
mean,13269.986924,4.726342e+04,8.464168e+04
std,38861.887034,1.360137e+05,7.166228e+04
min,63.000000,8.600000e+01,1.273697e+04
25%,1160.000000,4.273000e+03,4.303295e+04
50%,3288.000000,1.301700e+04,6.653278e+04
75%,8809.000000,3.362100e+04,9.753169e+04
max,527935.000000,1.830284e+06,1.234769e+06


In [ ]:
# Estadísticas de precipitación y consumo anual.
actual.groupby(["CVE_INEGI", "Año"]).sum().groupby("CVE_INEGI").mean().describe()["Pre"]

,Pre,Cons_us
count,2469.000000,2469.000000
mean,1107.132816,1062.087940
std,604.977786,567.664126
min,88.506901,145.939589
25%,695.096254,757.710660
50%,944.418709,944.626781
75%,1382.925214,1173.627261
max,4286.979643,6144.596894


In [ ]:
# Estadísticas de temperatura media anual.
actual.groupby(["CVE_INEGI"]).mean().describe()["Tmean"]

count    2469.000000
mean       22.416142
std         3.355948
min        15.178200
25%        19.961160
50%        21.875826
75%        24.857593
max        31.036904
Name: Tmean, dtype: float64

In [95]:
# Cambio porcentual promedio por estado.
df_2030 = pd.read_csv( path_data + "2030" + csv )
df_2030["Cons_us"] = df_2030["Consumo_1*"] / df_2030["Usuarios_1*"]

df_2030_mun = df_2030.groupby(["CVE_INEGI"]).mean()[
    ["CVE_ENT", "Usuarios_1*", "Cons_us"] ]
actual_mun = actual.groupby(["CVE_INEGI"]).mean()[
    ["CVE_ENT", "Usuarios_1*", "Cons_us"] ]

df_2030_mun[["Usuarios_1*", "Cons_us"]] = ( ( 
    df_2030_mun[["Usuarios_1*", "Cons_us"]]
    / actual_mun[["Usuarios_1*", "Cons_us"]] - 1) * 100 )

df_2030_mun = df_2030_mun.groupby("CVE_ENT").mean()
df_2030_mun["Estado"] = actual["NOM_ENT"].unique()
df_2030_mun["Cons_us"] = df_2030_mun["Cons_us"].apply(lambda x: f"{x:.0f}")
df_2030_mun["Usuarios_1*"] = df_2030_mun["Usuarios_1*"].apply(lambda x: f"{x:.0f}")

# Cambio porcentual promedio por estado.
df_2030_mun

,Usuarios_1*,Cons_us,Estado
CVE_ENT,,,
1.0,175,35,Aguascalientes
2.0,66,33,Baja California
3.0,66,-9,Baja California Sur
4.0,229,57,Campeche
5.0,140,13,Coahuila de Zaragoza
6.0,62,40,Colima
7.0,71,37,Chiapas
8.0,88,29,Chihuahua
9.0,55,34,Ciudad de MÃ©xico
